In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import copy
import random

from TrainingAlgorithm import TrainingAlgo
from DataPreprocess import create_dataloader
from model import TwoLayerNN

In [6]:
def All_r_LG_UA_w_LG_UA(model=None,
                        reg_params=None,
                        weight_params=None,
                        l2_lambda=0.001,
                        l1_lambda=0.001,
                        k=1,
                        p=50):
    
    prune_index = 0
    
    while not k>p:
        prune_index = random.randint(0, 127)
        
        train_algo.multiclass_regularization(epochs=reg_params['epochs'],
                                            model=model,
                                            optimizer=reg_params['optimizer'],
                                            loss_threshold=reg_params['loss_threshold'],
                                            eta_threshold=reg_params['eta_threshold'],
                                            l2_lambda=l2_lambda,
                                            l1_lambda=l1_lambda)
        
        saved_model = copy.deepcopy(model)
        
        model.del_neuron(index=prune_index)
        
        situation = train_algo.multiclass_weight_tuning(epochs=weight_params['epochs'],
                                                        model=model,
                                                        optimizer=weight_params['optimizer'],
                                                        loss_threshold=weight_params['loss_threshold'],
                                                        eta_threshold=weight_params['eta_threshold'],
                                                        LG=False)
        
        if situation == 'Unacceptable':
            print('All : Restore Network')
            model = saved_model
            k += 1
        
        elif situation == 'Acceptable':
            print('Pruning Works!')
            p -= 1
    
    print('finish training')

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TwoLayerNN(12, 128, 2, batch_norm=True, dropout=True)
train_loader, val_loader = create_dataloader()

# parameters for different task
reg_optimizer = optim.Adam(model.parameters(), lr=1e-3)
reg_loss_threshold = 0.6
reg_eta_threshold = 1e-6
reg_epochs = None

weight_optimizer = optim.Adam(model.parameters(), lr=1e-3)
weight_loss_threshold = 0.3
weight_eta_threshold = 1e-6
weight_epochs = 10

reg = {'optimizer': reg_optimizer,
       'loss_threshold': reg_loss_threshold,
       'eta_threshold': reg_eta_threshold,
       'epochs': reg_epochs}

weight = {'optimizer': weight_optimizer,
          'loss_threshold': weight_loss_threshold,
          'eta_threshold': weight_eta_threshold,
          'epochs': weight_epochs}

criterion = nn.CrossEntropyLoss()
train_algo = TrainingAlgo(train_loader, val_loader, criterion, device)
All_r_LG_UA_w_LG_UA(model, reg, weight, l2_lambda=0.001, l1_lambda=0.001, p=50)

--------initializing regularization--------
max loss:0.9564297795295715 > threshold0.6, stop training.
[ 1 ] | train_loss = 0.69385, train_acc = 0.57292, val_loss = 0.46153, val_acc = 0.63542
--------initializing weight tuning--------
[ 1/10 ] | train_loss = 0.60698, train_acc = 0.67188, val_loss = 0.54135, val_acc = 0.64583
[ 2/10 ] | train_loss = 0.60667, train_acc = 0.66667, val_loss = 0.70933, val_acc = 0.63542
[ 3/10 ] | train_loss = 0.60487, train_acc = 0.68229, val_loss = 0.52133, val_acc = 0.61458
[ 4/10 ] | train_loss = 0.59438, train_acc = 0.67969, val_loss = 0.58470, val_acc = 0.61458
[ 5/10 ] | train_loss = 0.59354, train_acc = 0.65625, val_loss = 0.52172, val_acc = 0.64583
[ 6/10 ] | train_loss = 0.58807, train_acc = 0.69271, val_loss = 0.56014, val_acc = 0.63542
learning rate < threshold
[ 7/10 ] | train_loss = 0.60688, train_acc = 0.67969, val_loss = 0.51068, val_acc = 0.62500
All : Restore Network
--------initializing regularization--------
max loss:0.7625904679298401 >